In [1]:
import numpy as np
import pandas as pd
import random
import datetime
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from datetime import date

In [2]:
#Creating the dataset

#features

"""
Weather features:
1. Temperature (Celcius) (-20 to 25)
2. Humidity (%) (relative) (0 to 100)
3. Probability of Precipitation (%) (0 to 100)
4. Wind speed (mph)

User features:
1. Age (15 to 60)
2. Weight (kg)
3. Sex
4. Height (cm)
5. Fat percentage
6. BMI
"""

#output
"""
output will be in the form of a boolean array with each index presenting different clothing articles
'thermal' : 0, #(both top and bottom)
'hoodie' : 1,
'fleece' : 2,
'wool' : 3,
'light_down' : 4,
'thick_down' : 5,
'wind_breaker' : 6,
'umbrella' : 7,
'winter_boots': 8
"""

"\noutput will be in the form of a boolean array with each index presenting different clothing articles\n'thermal' : 0, #(both top and bottom)\n'hoodie' : 1,\n'fleece' : 2,\n'wool' : 3,\n'light_down' : 4,\n'thick_down' : 5,\n'wind_breaker' : 6,\n'umbrella' : 7,\n'winter_boots': 8\n"

In [3]:
#smart generator for people's height, weight, age, sex
def peopledatagenerator():
    sex = random.randint(0,1)
    age = random.randint(15, 60)
    #stats from (United states)
    #https://en.wikipedia.org/wiki/Average_human_height_by_country
    #https://en.wikipedia.org/wiki/Human_body_weight
    #male
    if sex == 0:
        height = np.random.normal(loc = 175.3, scale = 6.35)
        weight = np.random.normal(loc = 90.6, scale = 15)
    #female
    else:
        height = np.random.normal(loc = 161.3, scale = 5.59)
        weight = np.random.normal(loc = 77.5, scale = 15)
        
    return sex, age, height, weight




In [4]:
#cold resistance calculation
male_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}


female_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}

# cold resistance categories:
# low: 0 - 24
# mid-low: 25 - 49
# mid-high: 50 - 74
# high: 75 - 99
cold_resistance_scores = {
    'low' : 25,
    'mid-low' : 50,
    'mid-high' : 75,
    'high' : 100
}


def coldresistance(sex, age, fatpercentage, BMI):
    #female case
    if sex == 1:
        if (age <= 15 or age >= 50):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 16 <= age <= 25:
            age_score = cold_resistance_scores['mid-high']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-low']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 14):
            fatpercentage_score = cold_resistance_scores['low']
        elif (14 < fatpercentage <= 20):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (20 < fatpercentage <= 25): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
      
    #men
    else:
        if (age <= 12 or age >= 50):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 16 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 8):
            fatpercentage_score = cold_resistance_scores['low']
        elif (8 < fatpercentage <= 15):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (15 < fatpercentage <= 24): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
            
    #female case
    if sex == 1:
        cold_resistance = age_score/100*female_weights['age'] + bmi_score/100*female_weights['bmi'] + fatpercentage_score/100*female_weights['fatpercentage']
    
    #male case
    elif sex == 0:
        cold_resistance = age_score/100*male_weights['age'] + bmi_score/100*male_weights['bmi'] + fatpercentage_score/100*male_weights['fatpercentage']
    
    return cold_resistance

In [5]:
clothesmap = {
    'thermal' : 0, #(both top and bottom)
    'hoodie' : 1,
    'fleece' : 2,
    'wool' : 3,
    'light_down' : 4,
    'thick_down' : 5,
    'wind_breaker' : 6,
    'umbrella' : 7,
    'winter_boots': 8
}

heatmap = {
    'thermal' : 30, #(both top and bottom)
    'hoodie' : 10,
    'fleece' : 20,
    'wool' : 40,
    'light_down' : 50,
    'thick_down' : 60
}



In [6]:
def findlowestheatscore(map):
    lowest = 100000
    for i in map:
        if map[i] < lowest:
            lowest = map[i]
    
    return lowest

#temperature range is from -20 to 25
def predictusingheatscore(temperature, cold_resistance, heatmap, windspeed, precipitation):
    #print(temperature)
    prediction = [False] * len(clothesmap.keys())
    
    #wind breaker and umbrella are independent of cold resistance
    if windspeed >= 25:
        prediction[clothesmap['wind_breaker']] = True
    
    if precipitation >= 50:
        prediction[clothesmap['umbrella']] = True
        
    if precipitation >= 50 and temperature <= 5:
        prediction[clothesmap['winter_boots']] = True
    
    #calulation layers required
    lowest_heatscore = findlowestheatscore(heatmap)

    
    #map using linear equation (-20,300) to (25, 0), x is temperature y is insulation needed
    insulation_calculation = -1 * (300/45) * temperature + (-1) * (300/45)* (-25)
    
    insulation_required = insulation_calculation - cold_resistance
    
    if insulation_required < lowest_heatscore:
        #dont need put on any extra layers if insulation required is lower than the lowest_heatscore
        return prediction
    
    else:
        result = []
        
        while insulation_required > 0:
            smallest_difference = 100000
            best_cloth = ''
            #hardstop at 5 layers
            if len(result) == 6:
                break
            
            for clothes in heatmap:
                #avoid repeats
                if clothes in result: continue
                difference = abs(insulation_required - heatmap[clothes])
                if difference < smallest_difference:
                    smallest_difference = difference
                    best_cloth = clothes
            
            result.append(best_cloth)
            insulation_required = insulation_required - heatmap[best_cloth]
            
        for ite in result:
            prediction[clothesmap[ite]] = True
            
        return prediction
        

In [7]:
#returns the features and output dataframes for ML
def generateDataset(count):
    from datetime import date
    data = []
    currentdate = date.today()
    for i in range(count):
        date = (currentdate - datetime.timedelta(i)).strftime("%d/%m/%Y") #display in day/month/year
        temperature = random.randint(0, 45) - 20
        humidity = random.randint(0, 100)
        precipitation = random.randint(0, 100)
        windspeed = random.randint(0, 40) #mph
        #age = random.randint(15, 60)
        #weight = random.randint(40, 120) #kg
        #height = random.randint(110, 200) #cm
        #sex = random.randint(0,1) # 0:M 1:F


        sex, age, height, weight = peopledatagenerator()

        #fat percentage formula from
        #https://www.gaiam.com/blogs/discover/how-to-calculate-your-ideal-body-fat-percentage

        BMI = weight/(height**2)*10000
        if sex == 0:
            fatpercentage = (1.20 * BMI) + (0.23 * age) - 16.2

        elif sex == 1:
            fatpercentage = (1.20 * BMI) + (0.23 * age) - 5.4

        #adjustment for negative fatpercentage
        if fatpercentage < 0:
            fatpercentage = 1

        datapoint = (date, temperature, humidity, precipitation, windspeed, age, weight, height, sex, fatpercentage, BMI)
        data.append(datapoint)
        
        
    columnstring = 'date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi'
    column = columnstring.split(',')
    df = pd.DataFrame(data, columns = column)
    
    
    df['cold_resistance'] = df.apply(lambda row : coldresistance(row['sex'], row['age'], row['fatpercentage'], row['bmi']), axis = 1)
    
    for article in clothesmap:
        df[article] = df.apply(lambda row : predictusingheatscore(row['temperature'], row['cold_resistance'], heatmap,\
                                                                  row['windspeed'], row['precipitation'])[clothesmap[article]], axis = 1)
    
    features = df[['temperature', 'humidity', 'precipitation', 'windspeed', 'age', 'weight', 'height', 'sex', 'fatpercentage', 'bmi', 'cold_resistance' ]]
    output = df[clothesmap.keys()]
    return features, output
    

In [8]:
#features, output = generateDataset(10**5)

In [11]:
#features.tail()

,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance
99995,-5,92,66,16,51,105.583091,179.214628,0,34.978338,32.873615,70.0
99996,-6,40,2,24,24,100.334839,182.985370,0,25.278392,29.965327,80.0
99997,21,42,1,36,15,49.368867,150.780467,1,24.108190,21.715158,55.0
99998,-20,86,69,14,16,88.860260,185.923912,0,18.327383,25.706153,72.5
99999,-9,71,21,40,27,87.854938,180.793197,0,22.263975,26.878313,92.5


In [12]:
#output.tail()

,thermal,hoodie,fleece,wool,light_down,thick_down,wind_breaker,umbrella,winter_boots
99995,False,True,True,False,True,True,False,True,True
99996,False,False,True,False,True,True,False,False,False
99997,False,False,False,False,False,False,True,False,False
99998,True,True,True,True,True,True,False,True,True
99999,False,True,True,False,True,True,True,False,False
